In [128]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [129]:
plt.rcParams.update({"text.usetex": False,
                     "font.size": 20,
                     "pdf.fonttype": 42,
                     "ps.fonttype": 42})

In [130]:
target_latency = 160

In [ ]:
local_df = pd.read_csv("../measurements/local_long/map.csv")

root_folder = "../measurements/real_bw_test_15fps/"

map_df = pd.read_csv(root_folder+"map.csv")
cha_df = pd.read_csv(root_folder+"characteristic.csv")
mag_df = pd.read_csv(root_folder+"manager.csv")
tim_df = pd.read_csv(root_folder+"time.csv")

cal_time = []
for i in range(len(tim_df)):
    if tim_df["technique"][i] == 2:
        cal_time.append(tim_df["compression_time"][i]/32)
    elif tim_df["technique"][i] == 3:
        cal_time.append(tim_df["compression_time"][i]-50)
    else:
        cal_time.append(tim_df["compression_time"][i])
tim_df["compression_time"] = cal_time

tim_df["transmission_time"] = cha_df["datasize_est"]*8/cha_df["consumed_bw"]*1000
tim_df["overall_est"] = tim_df["model_head_time"]*10+tim_df["model_tail_time"]+tim_df["framework_head_time"]+tim_df["framework_tail_time"] + tim_df["compression_time"]+tim_df["decompression_time"]+ tim_df["transmission_time"]

In [132]:
drops = []

for i in range(400):
    try:
        d = mag_df[mag_df["frame_id"]==i+1]["drop"].tolist()[0]
        if d >0.4:
            d=0.4
        drops.append(d)
    except:
        drops.append(drops[-1])


df = pd.DataFrame()
df["traget_drops"] = drops

IndexError: list index out of range

In [ ]:
df["map_drop"] = local_df["map"] - map_df["map"]
df["sen_drop"] = local_df["sensitivity"] - map_df["sensitivity"]

In [ ]:
df["bandwidth"] = cha_df["bandwidth"]
df["bw_com"] = cha_df["consumed_bw"]
df["overall_time"] = tim_df["overall_est"]

In [ ]:
df = df[200:]

In [ ]:
df

,traget_drops,map_drop,sen_drop,bandwidth,bw_com,overall_time
200,0.4,0.166667,0.333333,6000000.0,333442.622951,147.859531
201,0.4,0.229167,0.333333,6000000.0,396000.000000,148.874602
202,0.4,0.055556,0.200000,6000000.0,345836.065574,148.232429
203,0.4,0.187500,0.000000,6000000.0,768983.606557,148.976043
204,0.4,0.178571,0.000000,6000000.0,386754.098361,147.673004
...,...,...,...,...,...,...
395,0.4,0.100000,0.142857,6000000.0,325377.049180,149.787179
396,0.4,0.037500,0.142857,6000000.0,333836.065574,152.086218
397,0.4,0.350000,0.285714,6000000.0,343278.688525,150.172107
398,0.4,0.200000,0.200000,6000000.0,338360.655738,150.541318


In [ ]:
good_df = df[df["map_drop"]<df["traget_drops"]]
good_df= good_df[good_df["sen_drop"]<good_df["traget_drops"]]
good_df= good_df[good_df["bw_com"]<good_df["bandwidth"]]
good_df= good_df[good_df["overall_time"]<target_latency]

In [ ]:
len(good_df)/200

0.96